In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os

In [2]:
# Load the dataset
a1 = pd.read_excel("case_study1.xlsx")
a2 = pd.read_excel("case_study2.xlsx")

In [3]:
df1 = a1.copy()
df2 = a2.copy()

In [4]:
# Remove nulls
df1 = df1.loc[df1['Age_Oldest_TL'] != -99999]

columns_to_be_removed = []

for i in df2.columns:
    if df2.loc[df2[i] == -99999].shape[0] > 10000:
        columns_to_be_removed .append(i)
columns_to_be_removed

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [5]:
df2 = df2.drop(columns_to_be_removed, axis =1)
for i in df2.columns:
    df2 = df2.loc[ df2[i] != -99999 ]
df2

,PROSPECTID,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,549,11,29,0,0,0,0,0,21,...,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,47,0,0,0,0,0,0,0,0,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,302,9,25,1,9,8,0,0,10,...,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
4,5,583,0,0,0,0,0,0,0,53,...,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
5,6,245,14,270,0,0,0,13,11,5,...,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,15,2,24,0,0,0,0,0,0,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,650,P4
51332,51333,57,0,0,0,0,0,0,0,6,...,0.0,0.0,0.000,0.0,0,0,others,others,702,P1
51333,51334,32,0,0,0,0,0,0,0,0,...,1.0,0.0,1.000,0.0,0,0,ConsumerLoan,others,661,P3
51334,51335,58,0,0,0,0,0,0,0,0,...,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,others,686,P2


In [6]:
df1

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.00,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.00,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.00,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.00,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.00,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51331,51332,3,0,3,1,0,0.333,0.00,1.000,0.000,...,0,2,0,0,0,0,3,1,24,5
51332,51333,4,2,2,0,1,0.000,0.25,0.500,0.500,...,0,2,0,0,0,2,2,0,74,7
51333,51334,2,1,1,1,1,0.500,0.50,0.500,0.500,...,0,2,0,0,0,0,2,0,9,5
51334,51335,2,1,1,0,0,0.000,0.00,0.500,0.500,...,0,2,0,0,0,0,2,0,15,8


In [7]:
# Checking common column names
for i in list(df1.columns):
    if i in list(df2.columns):
        print (i)

PROSPECTID


In [8]:
# Merge the two dataframes, inner join so that no nulls are present
df = pd. merge ( df1, df2, how ='inner', left_on = ['PROSPECTID'], right_on = ['PROSPECTID'] )

In [9]:
df.shape

(42064, 79)

In [10]:
# check how many columns are categorical
for i in df.columns:
    if df[i].dtype == 'object':
        print(i)

MARITALSTATUS
EDUCATION
GENDER
last_prod_enq2
first_prod_enq2
Approved_Flag


In [11]:
# Chi-square test
for i in ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']:
    chi2, pval, _, _ = chi2_contingency(pd.crosstab(df[i], df['Approved_Flag']))
    print(i, '---', pval)
# Since all the categorical features have pval <=0.05, we will accept all

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


In [12]:
# VIF for numerical columns
numeric_columns = []
for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [13]:
# VIF sequentially check
vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0

In [14]:
import warnings
warnings.filterwarnings('ignore')
for i in range (0,total_columns):
    
    vif_value = variance_inflation_factor(vif_data, column_index)
    print (column_index,'---',vif_value)
    
    
    if vif_value <= 6:
        columns_to_be_kept.append( numeric_columns[i] )
        column_index = column_index+1
    
    else:
        vif_data = vif_data.drop([ numeric_columns[i] ] , axis=1)

0 --- inf
0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000336
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735
2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.8328007921530785
3 --- 6.099653381646723
3 --- 5.581352009642762
4 --- 1.9855843530987785
5 --- inf
5 --- 4.809538302819343
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.384346405965583
7 --- 3.0646584155234238
8 --- 2.898639771299253
9 --- 4.377876915347324
10 --- 2.207853583695844
11 --- 4.916914200506864
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721434
15 --- inf
15 --- 7.380634506427232
15 --- 1.421005001517573
16 --- 8.083255010190323
16 --- 1.624122752404011
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.825857047132431
18 --- 1.5080839450032666
19 --- 2.172088834824578
20 --- 2.6233975535272283
21 --- 2.2959970812106176
22 --- 7.360578319196446
22 --- 2.1602387773102554
23 --- 2.8686288267891475
24 -

In [15]:
# check Anova for columns_to_be_kept 

from scipy.stats import f_oneway

columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])  
    b = list(df['Approved_Flag'])  
    
    group_P1 = [value for value, group in zip(a, b) if group == 'P1']
    group_P2 = [value for value, group in zip(a, b) if group == 'P2']
    group_P3 = [value for value, group in zip(a, b) if group == 'P3']
    group_P4 = [value for value, group in zip(a, b) if group == 'P4']


    f_statistic, p_value = f_oneway(group_P1, group_P2, group_P3, group_P4)

    if p_value <= 0.05:
        columns_to_be_kept_numerical.append(i)

In [16]:
# feature selection is done for cat and num features
# listing all the final features
features = columns_to_be_kept_numerical + ['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']
df = df[features + ['Approved_Flag']]

In [17]:
# Label encoding for the categorical features
['MARITALSTATUS', 'EDUCATION', 'GENDER' , 'last_prod_enq2' ,'first_prod_enq2']

['MARITALSTATUS', 'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

In [18]:
df['MARITALSTATUS'].unique()    
df['EDUCATION'].unique()
df['GENDER'].unique()
df['last_prod_enq2'].unique()
df['first_prod_enq2'].unique()
# Ordinal feature -- EDUCATION
# SSC            : 1
# 12TH           : 2
# GRADUATE       : 3
# UNDER GRADUATE : 3
# POST-GRADUATE  : 4
# OTHERS         : 1
# PROFESSIONAL   : 3



array(['PL', 'ConsumerLoan', 'others', 'AL', 'HL', 'CC'], dtype=object)

In [19]:
df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']]              = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']]             = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']]         = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']]   = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']]    = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']]           = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']]     = 3
df['EDUCATION'].value_counts()
df['EDUCATION'] = df['EDUCATION'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

In [20]:
df_encoded = pd.get_dummies(df, columns=['MARITALSTATUS','GENDER', 'last_prod_enq2' ,'first_prod_enq2'])

df_encoded.info()
k = df_encoded.describe()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [21]:
# Machine Learing model fitting
# Data processing

# 1. Random Forest
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators = 200, random_state=42)
rf_classifier.fit(x_train, y_train)
y_pred = rf_classifier.predict(x_test)

In [23]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)


Accuracy: 0.7636990372043266


In [24]:
for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
   

Class p1:
Precision: 0.8370457209847597
Recall: 0.7041420118343196
F1 Score: 0.7648634172469202
Class p2:
Precision: 0.7957519116397621
Recall: 0.9282457879088206
F1 Score: 0.856907593778591
Class p3:
Precision: 0.4423380726698262
Recall: 0.21132075471698114
F1 Score: 0.28600612870275793
Class p4:
Precision: 0.7178502879078695
Recall: 0.7269193391642371
F1 Score: 0.7223563495895703


In [25]:
# 2. xgboost

import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)

In [26]:
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [27]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)


In [28]:
xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f'Accuracy: {accuracy:.2f}')
print ()

precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Accuracy: 0.78

Class p1:
Precision: 0.823906083244397
Recall: 0.7613412228796844
F1 Score: 0.7913890312660175

Class p2:
Precision: 0.8255418233924413
Recall: 0.913577799801784
F1 Score: 0.8673315769665035

Class p3:
Precision: 0.4756380510440835
Recall: 0.30943396226415093
F1 Score: 0.37494284407864653

Class p4:
Precision: 0.7342386032977691
Recall: 0.7356656948493683
F1 Score: 0.7349514563106796



In [29]:
# 3. Decision Tree
from sklearn.tree import DecisionTreeClassifier

y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [30]:
dt_model = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_model.fit(x_train, y_train)
y_pred = dt_model.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print ()
print(f"Accuracy: {accuracy:.2f}")
print ()


Accuracy: 0.71



In [31]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()


Class p1:
Precision: 0.7211822660098522
Recall: 0.7218934911242604
F1 Score: 0.7215377033021193

Class p2:
Precision: 0.8102005061319837
Recall: 0.8249752229930625
F1 Score: 0.8175211156943626

Class p3:
Precision: 0.3409090909090909
Recall: 0.3283018867924528
F1 Score: 0.3344867358708189

Class p4:
Precision: 0.6517766497461929
Recall: 0.6239067055393586
F1 Score: 0.637537239324727



In [32]:
# xgboost is giving me best results
# We will further finetune it


# Apply standard scaler 

from sklearn.preprocessing import StandardScaler

columns_to_be_scaled = ['Age_Oldest_TL','Age_Newest_TL','time_since_recent_payment',
'max_recent_level_of_deliq','recent_level_of_deliq',
'time_since_recent_enq','NETMONTHLYINCOME','Time_With_Curr_Empr']


In [33]:
for i in columns_to_be_scaled:
    column_data = df_encoded[i].values.reshape(-1, 1)
    scaler = StandardScaler()
    scaled_column = scaler.fit_transform(column_data)
    df_encoded[i] = scaled_column


In [34]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',  num_class=4)

In [35]:
y = df_encoded['Approved_Flag']
x = df_encoded. drop ( ['Approved_Flag'], axis = 1 )


label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

In [37]:
xgb_classifier.fit(x_train, y_train)
y_pred = xgb_classifier.predict(x_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.78


In [38]:
precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_pred)

for i, v in enumerate(['p1', 'p2', 'p3', 'p4']):
    print(f"Class {v}:")
    print(f"Precision: {precision[i]}")
    print(f"Recall: {recall[i]}")
    print(f"F1 Score: {f1_score[i]}")
    print()
    

Class p1:
Precision: 0.823906083244397
Recall: 0.7613412228796844
F1 Score: 0.7913890312660175

Class p2:
Precision: 0.8255418233924413
Recall: 0.913577799801784
F1 Score: 0.8673315769665035

Class p3:
Precision: 0.4756380510440835
Recall: 0.30943396226415093
F1 Score: 0.37494284407864653

Class p4:
Precision: 0.7342386032977691
Recall: 0.7356656948493683
F1 Score: 0.7349514563106796



In [39]:
# Hyperparameter tuning in xgboost
from sklearn.model_selection import GridSearchCV
x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

# Define the XGBClassifier with the initial set of hyperparameters
xgb_model = xgb.XGBClassifier(objective='multi:softmax', num_class=4)


In [40]:
# Define the parameter grid for hyperparameter tuning

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
}

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_class=4,
                                     num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy')

In [41]:
# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Evaluate the model with the best hyperparameters on the test set
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test, y_test)
print("Test Accuracy:", accuracy)

Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}
Test Accuracy: 0.7811719957209081


In [42]:
# Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 3, 'n_estimators': 200}

# Based on risk appetite of the bank, you will suggest P1,P2,P3,P4 to the business end user

# # Hyperparameter tuning for xgboost (Used in the session)
param_grid = {
  'colsample_bytree': [0.1, 0.3, 0.5, 0.7, 0.9],
  'learning_rate'   : [0.001, 0.01, 0.1, 1],
  'max_depth'       : [3, 5, 8, 10],
  'alpha'           : [1, 10, 100],
  'n_estimators'    : [10,50,100]
}

index = 0

answers_grid = {
    'combination'       :[],
    'train_Accuracy'    :[],
    'test_Accuracy'     :[],
    'colsample_bytree'  :[],
    'learning_rate'     :[],
    'max_depth'         :[],
    'alpha'             :[],
    'n_estimators'      :[]
    }



In [43]:
for colsample_bytree in param_grid['colsample_bytree']:
  for learning_rate in param_grid['learning_rate']:
    for max_depth in param_grid['max_depth']:
      for alpha in param_grid['alpha']:
          for n_estimators in param_grid['n_estimators']:
             
              index = index + 1
             
              model = xgb.XGBClassifier(objective='multi:softmax',  
                                       num_class=4,
                                       colsample_bytree = colsample_bytree,
                                       learning_rate = learning_rate,
                                       max_depth = max_depth,
                                       alpha = alpha,
                                       n_estimators = n_estimators)
               
              y = df_encoded['Approved_Flag']
              x = df_encoded.drop(['Approved_Flag'], axis = 1)

              label_encoder = LabelEncoder()
              y_encoded = label_encoder.fit_transform(y)

              x_train, x_test, y_train, y_test = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

              model.fit(x_train, y_train)

              y_pred_train = model.predict(x_train)
              y_pred_test = model.predict(x_test)

              train_accuracy =  accuracy_score(y_train, y_pred_train)
              test_accuracy  =  accuracy_score(y_test , y_pred_test)

              answers_grid['combination'].append(index)
              answers_grid['train_Accuracy'].append(train_accuracy)
              answers_grid['test_Accuracy'].append(test_accuracy)
              answers_grid['colsample_bytree'].append(colsample_bytree)
              answers_grid['learning_rate'].append(learning_rate)
              answers_grid['max_depth'].append(max_depth)
              answers_grid['alpha'].append(alpha)
              answers_grid['n_estimators'].append(n_estimators)

              print(f"Combination {index}")
              print(f"colsample_bytree: {colsample_bytree}, learning_rate: {learning_rate}, max_depth: {max_depth}, alpha: {alpha}, n_estimators: {n_estimators}")
              print(f"Train Accuracy: {train_accuracy:.2f}")
              print(f"Test Accuracy : {test_accuracy :.2f}")
              print("-" * 30)



Combination 1
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.61
------------------------------
Combination 2
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 3
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 100
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 4
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 5
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 6
colsample_bytree: 0.1, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 100
Train Accuracy: 0.61
Test Accu

Combination 48
colsample_bytree: 0.1, learning_rate: 0.01, max_depth: 5, alpha: 1, n_estimators: 100
Train Accuracy: 0.62
Test Accuracy : 0.62
------------------------------
Combination 49
colsample_bytree: 0.1, learning_rate: 0.01, max_depth: 5, alpha: 10, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.61
------------------------------
Combination 50
colsample_bytree: 0.1, learning_rate: 0.01, max_depth: 5, alpha: 10, n_estimators: 50
Train Accuracy: 0.61
Test Accuracy : 0.61
------------------------------
Combination 51
colsample_bytree: 0.1, learning_rate: 0.01, max_depth: 5, alpha: 10, n_estimators: 100
Train Accuracy: 0.62
Test Accuracy : 0.61
------------------------------
Combination 52
colsample_bytree: 0.1, learning_rate: 0.01, max_depth: 5, alpha: 100, n_estimators: 10
Train Accuracy: 0.61
Test Accuracy : 0.60
------------------------------
Combination 53
colsample_bytree: 0.1, learning_rate: 0.01, max_depth: 5, alpha: 100, n_estimators: 50
Train Accuracy: 0.61
Test 

Combination 96
colsample_bytree: 0.1, learning_rate: 0.1, max_depth: 8, alpha: 10, n_estimators: 100
Train Accuracy: 0.73
Test Accuracy : 0.72
------------------------------
Combination 97
colsample_bytree: 0.1, learning_rate: 0.1, max_depth: 8, alpha: 100, n_estimators: 10
Train Accuracy: 0.62
Test Accuracy : 0.61
------------------------------
Combination 98
colsample_bytree: 0.1, learning_rate: 0.1, max_depth: 8, alpha: 100, n_estimators: 50
Train Accuracy: 0.67
Test Accuracy : 0.66
------------------------------
Combination 99
colsample_bytree: 0.1, learning_rate: 0.1, max_depth: 8, alpha: 100, n_estimators: 100
Train Accuracy: 0.71
Test Accuracy : 0.70
------------------------------
Combination 100
colsample_bytree: 0.1, learning_rate: 0.1, max_depth: 10, alpha: 1, n_estimators: 10
Train Accuracy: 0.64
Test Accuracy : 0.63
------------------------------
Combination 101
colsample_bytree: 0.1, learning_rate: 0.1, max_depth: 10, alpha: 1, n_estimators: 50
Train Accuracy: 0.72
Test Ac

Combination 144
colsample_bytree: 0.1, learning_rate: 1, max_depth: 10, alpha: 100, n_estimators: 100
Train Accuracy: 0.77
Test Accuracy : 0.76
------------------------------
Combination 145
colsample_bytree: 0.3, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 10
Train Accuracy: 0.67
Test Accuracy : 0.67
------------------------------
Combination 146
colsample_bytree: 0.3, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 50
Train Accuracy: 0.67
Test Accuracy : 0.66
------------------------------
Combination 147
colsample_bytree: 0.3, learning_rate: 0.001, max_depth: 3, alpha: 1, n_estimators: 100
Train Accuracy: 0.66
Test Accuracy : 0.66
------------------------------
Combination 148
colsample_bytree: 0.3, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 10
Train Accuracy: 0.67
Test Accuracy : 0.67
------------------------------
Combination 149
colsample_bytree: 0.3, learning_rate: 0.001, max_depth: 3, alpha: 10, n_estimators: 50
Train Accuracy: 0.67

Combination 191
colsample_bytree: 0.3, learning_rate: 0.01, max_depth: 5, alpha: 1, n_estimators: 50
Train Accuracy: 0.70
Test Accuracy : 0.69
------------------------------
Combination 192
colsample_bytree: 0.3, learning_rate: 0.01, max_depth: 5, alpha: 1, n_estimators: 100
Train Accuracy: 0.71
Test Accuracy : 0.70
------------------------------
Combination 193
colsample_bytree: 0.3, learning_rate: 0.01, max_depth: 5, alpha: 10, n_estimators: 10
Train Accuracy: 0.69
Test Accuracy : 0.69
------------------------------
Combination 194
colsample_bytree: 0.3, learning_rate: 0.01, max_depth: 5, alpha: 10, n_estimators: 50
Train Accuracy: 0.70
Test Accuracy : 0.69
------------------------------
Combination 195
colsample_bytree: 0.3, learning_rate: 0.01, max_depth: 5, alpha: 10, n_estimators: 100
Train Accuracy: 0.70
Test Accuracy : 0.69
------------------------------
Combination 196
colsample_bytree: 0.3, learning_rate: 0.01, max_depth: 5, alpha: 100, n_estimators: 10
Train Accuracy: 0.68
T

...

Combination 522
colsample_bytree: 0.7, learning_rate: 0.1, max_depth: 5, alpha: 100, n_estimators: 100
Train Accuracy: 0.78
Test Accuracy : 0.77
------------------------------
Combination 523
colsample_bytree: 0.7, learning_rate: 0.1, max_depth: 8, alpha: 1, n_estimators: 10
Train Accuracy: 0.80
Test Accuracy : 0.77
------------------------------
Combination 524
colsample_bytree: 0.7, learning_rate: 0.1, max_depth: 8, alpha: 1, n_estimators: 50
Train Accuracy: 0.84
Test Accuracy : 0.78
------------------------------
Combination 525
colsample_bytree: 0.7, learning_rate: 0.1, max_depth: 8, alpha: 1, n_estimators: 100
Train Accuracy: 0.88
Test Accuracy : 0.78
------------------------------
Combination 526
colsample_bytree: 0.7, learning_rate: 0.1, max_depth: 8, alpha: 10, n_estimators: 10
Train Accuracy: 0.78
Test Accuracy : 0.76
------------------------------
Combination 527
colsample_bytree: 0.7, learning_rate: 0.1, max_depth: 8, alpha: 10, n_estimators: 50
Train Accuracy: 0.80
Test Acc

Combination 570
colsample_bytree: 0.7, learning_rate: 1, max_depth: 10, alpha: 1, n_estimators: 100
Train Accuracy: 1.00
Test Accuracy : 0.76
------------------------------
Combination 571
colsample_bytree: 0.7, learning_rate: 1, max_depth: 10, alpha: 10, n_estimators: 10
Train Accuracy: 0.83
Test Accuracy : 0.77
------------------------------
Combination 572
colsample_bytree: 0.7, learning_rate: 1, max_depth: 10, alpha: 10, n_estimators: 50
Train Accuracy: 0.93
Test Accuracy : 0.76
------------------------------
Combination 573
colsample_bytree: 0.7, learning_rate: 1, max_depth: 10, alpha: 10, n_estimators: 100
Train Accuracy: 0.96
Test Accuracy : 0.76
------------------------------
Combination 574
colsample_bytree: 0.7, learning_rate: 1, max_depth: 10, alpha: 100, n_estimators: 10
Train Accuracy: 0.78
Test Accuracy : 0.77
------------------------------
Combination 575
colsample_bytree: 0.7, learning_rate: 1, max_depth: 10, alpha: 100, n_estimators: 50
Train Accuracy: 0.78
Test Accura

Combination 617
colsample_bytree: 0.9, learning_rate: 0.01, max_depth: 3, alpha: 10, n_estimators: 50
Train Accuracy: 0.72
Test Accuracy : 0.72
------------------------------
Combination 618
colsample_bytree: 0.9, learning_rate: 0.01, max_depth: 3, alpha: 10, n_estimators: 100
Train Accuracy: 0.72
Test Accuracy : 0.72
------------------------------
Combination 619
colsample_bytree: 0.9, learning_rate: 0.01, max_depth: 3, alpha: 100, n_estimators: 10
Train Accuracy: 0.70
Test Accuracy : 0.70
------------------------------
Combination 620
colsample_bytree: 0.9, learning_rate: 0.01, max_depth: 3, alpha: 100, n_estimators: 50
Train Accuracy: 0.71
Test Accuracy : 0.71
------------------------------
Combination 621
colsample_bytree: 0.9, learning_rate: 0.01, max_depth: 3, alpha: 100, n_estimators: 100
Train Accuracy: 0.72
Test Accuracy : 0.71
------------------------------
Combination 622
colsample_bytree: 0.9, learning_rate: 0.01, max_depth: 5, alpha: 1, n_estimators: 10
Train Accuracy: 0.7

Combination 664
colsample_bytree: 0.9, learning_rate: 0.1, max_depth: 5, alpha: 100, n_estimators: 10
Train Accuracy: 0.74
Test Accuracy : 0.74
------------------------------
Combination 665
colsample_bytree: 0.9, learning_rate: 0.1, max_depth: 5, alpha: 100, n_estimators: 50
Train Accuracy: 0.77
Test Accuracy : 0.76
------------------------------
Combination 666
colsample_bytree: 0.9, learning_rate: 0.1, max_depth: 5, alpha: 100, n_estimators: 100
Train Accuracy: 0.78
Test Accuracy : 0.77
------------------------------
Combination 667
colsample_bytree: 0.9, learning_rate: 0.1, max_depth: 8, alpha: 1, n_estimators: 10
Train Accuracy: 0.80
Test Accuracy : 0.77
------------------------------
Combination 668
colsample_bytree: 0.9, learning_rate: 0.1, max_depth: 8, alpha: 1, n_estimators: 50
Train Accuracy: 0.84
Test Accuracy : 0.78
------------------------------
Combination 669
colsample_bytree: 0.9, learning_rate: 0.1, max_depth: 8, alpha: 1, n_estimators: 100
Train Accuracy: 0.88
Test A

Combination 712
colsample_bytree: 0.9, learning_rate: 1, max_depth: 10, alpha: 1, n_estimators: 10
Train Accuracy: 0.92
Test Accuracy : 0.76
------------------------------
Combination 713
colsample_bytree: 0.9, learning_rate: 1, max_depth: 10, alpha: 1, n_estimators: 50
Train Accuracy: 1.00
Test Accuracy : 0.76
------------------------------
Combination 714
colsample_bytree: 0.9, learning_rate: 1, max_depth: 10, alpha: 1, n_estimators: 100
Train Accuracy: 1.00
Test Accuracy : 0.76
------------------------------
Combination 715
colsample_bytree: 0.9, learning_rate: 1, max_depth: 10, alpha: 10, n_estimators: 10
Train Accuracy: 0.84
Test Accuracy : 0.77
------------------------------
Combination 716
colsample_bytree: 0.9, learning_rate: 1, max_depth: 10, alpha: 10, n_estimators: 50
Train Accuracy: 0.94
Test Accuracy : 0.77
------------------------------
Combination 717
colsample_bytree: 0.9, learning_rate: 1, max_depth: 10, alpha: 10, n_estimators: 100
Train Accuracy: 0.96
Test Accuracy :

In [44]:
answers_grid_df = pd.DataFrame(answers_grid)
answers_grid_df .to_excel ('hyperparameter0.xlsx', index=False)

In [45]:
model = xgb.XGBRegressor(objective='reg:squarederror',
                         colsample_bytree = 0.5,
                         learning_rate    = 0.1,
                         max_depth        = 8,
                         alpha            = 1,
                         n_estimators     = 100)
   
model.fit(x_train, y_train)

XGBRegressor(alpha=1, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=8, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [46]:
# Predicting on the testing set
y_pred = model.predict(x_test)

In [48]:
from sklearn.metrics import mean_squared_error

# Your existing code
r2 = r2_score(y_test, y_pred)
print('R-squared:', r2)

# Calculate RMSE using mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', rmse)

# Calculate the difference in standard deviation
stddev = y_test.std()
print('Stddev difference:', rmse / stddev)


R-squared: 0.7083624395364156
RMSE: 0.44792723374697513
Stddev difference: 0.5400347770871654
